# 데이터베이스에 저장되어 있는 댓글들을 기사별로 파일로 만들자

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

### Path

In [5]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    news_senti_outcome = '/Volumes/disk1/outcome_for_News_sentiment_analysis/'
    daumCommentsPath = '/Volumes/disk1/data/daum_Comments/'
    naverCommentsPath = '/Volumes/disk1/data/naver_Comments/'
    outcomeDaumCommentsPath = '/Volumes/disk1/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = '/Volumes/disk1/outcome_comments_for_naver/'
    outcome_predata = '/Volumes/disk1/pre_data_for_comments/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'
    news_senti_outcome = './outcome_for_News_sentiment_analysis/'
    daumCommentsPath = 'd:/data/daum_Comments/'
    naverCommentsPath = 'd:/data/naver_Comments/'
    outcomeDaumCommentsPath = 'd:/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = 'd:/outcome_comments_for_naver/'
    outcome_predata = 'd:/pre_data_for_comments/'

### News

In [6]:
os.listdir(news_senti_outcome)

['daum_news_sentiment_analysis.csv', 'naver_news_sentiment_analysis.csv']

In [7]:
# Daum
daumData = pd.read_csv(os.path.join(news_senti_outcome, 'daum_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
daumData['site'] = ['daum'] * daumData.shape[0]
#daumData.head()

#### 크롤링된 댓글수가 0개인 것을 찾아보자

In [8]:
daumData[daumData.number_of_crawled_comment == 0]

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a3c39e7588c132bc05289fb,뉴스,2017.12.21,노컷뉴스,17384,0,7,"[인터뷰] 서민 ""'문빠' 해악, 더 두고 볼 수 없었다""","- 낚싯배 사고 & 中 기자폭행 관련 '문빠' 반응, 도를 넘은 듯 - '보수 쪽도...","['정관용', '댓글', '낚싯배']","{'댓글', '문빠들', '폭행', '대통령', '낚싯배', '서민 교수', '정관용'}",95.0,13.0,negative,daum
5a64385d588c134738c38086,연예,2018.01.07,뉴스엔,0,0,45,"'언니네' 송은이 ""라이언전, 10년 전 박소현 맞선남으로 출연""",[뉴스엔 박수인 기자] 개그우먼 송은이가 작곡가 라이언전의 과거를 공개했다. 1월 ...,"['박소현', '언니네라디오', '김숙', '개그우먼', '게스트']","{'방송', '언니네', '작곡가 라이언전', '송은이', '민망', '박소현 맞선남'}",65.0,43.0,negative,daum
5a6472e8588c131b18d57abe,스포츠,2018.01.11,MK스포츠,0,0,38,"MLB 에이전트, 선수 샤워 장면 몰래 촬영해 '충격'",[매경닷컴 MK스포츠 김재호 기자] 다수의 메이저리그 선수를 고객으로 보유하고 있는...,NaN,"{'샤워', '촬영', '우드', '기준', '사장', '회사', '장면', '에이...",39.0,69.0,positive,daum


daum 뉴스를 크롤링한 데이터베이스에서는 댓글수가 0개인 뉴스가 3개 존재함.
> 뉴스 1건의 댓글수(글쓴이가 삭제한 댓글 포함)가 17,384건이지만, 크롤링된 댓글은 0개............  
>> 2017.12.21 뉴스. 발견은 2018년 3월 14일. 3달이나 지나서 댓글을 수집하면 삭제할 사람은 삭제했을 것. 
>> * 아쉽지만 데이터베이스에 댓글이 존재하는 것에 대해서만 진행하기로 함. 

### 새롭게 정의된 뉴스목록

In [9]:
reDaumData = daumData[daumData.number_of_crawled_comment != 0]

In [10]:
reDaumData.head()

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,903,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","['부작용', '비아그라', '아일랜드']","{'연기', '부작용', '남성들', '지역', '건강', '공장', '세보 효과'}",50.0,58.0,positive,daum
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"['커피', '낮잠', '음식물']","{'입냄새', '커피', '디스크', '점심 식사', '낮잠', '식후', '치아'...",81.0,27.0,negative,daum
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,807,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"['살인혐의', '철원', '검찰송치']","{'남편', '경찰', '철원', '아들', '진술', '범행', '주변', '앙심...",99.0,9.0,negative,daum
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,368,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\r\n신영자, 얼마를 어떻게 받았나 [...","['신영자', '갑질', '롯데백화점']","{'검찰', '신영자 이사장', '매장', '혐의', '유통업체', '네이처리퍼블릭...",77.0,31.0,negative,daum
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,367,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"['공범', '살인', '과자']","{'경찰', '유치장', '폭행', '쪽지', '과자', '혐의', '범행', '남...",102.0,6.0,negative,daum


### 댓글을 dataframe으로 만들어 저장. 
* 읽어들이는데 시간이 오래 걸림. 

In [11]:
import dask.dataframe as dd

In [12]:
# 다음
dd_daum = dd.from_pandas(reDaumData, npartitions=30)
def Daum_Comment(row):
    return bm.Make_Comments_File(daumCommentPath, row)
dd_daum.apply(Daum_Comment, axis = 1, meta = int).compute()

(96, 8)(107, 8)

(52, 8)
(93, 8)
(98, 8)
(134, 8)
(24, 8)
(195, 8)
(246, 8)
(554, 8)
(818, 8)
(1068, 8)
(39, 8)
(122, 8)
(138, 8)
(14, 8)
(4, 8)
(22, 8)
(66, 8)
(158, 8)(190, 8)

(132, 8)
(1410, 8)
(2241, 8)
(27, 8)
(55, 8)
(43, 8)
(52, 8)
(95, 8)
(134, 8)
(271, 8)
(415, 8)
(1219, 8)
(841, 8)
(585, 8)
(1732, 8)
(50, 8)
(50, 8)
(51, 8)
(94, 8)
(52, 8)
(101, 8)
(23, 8)
(102, 8)
(421, 8)
(41, 8)
(1096, 8)
(4172, 8)
(24, 8)
(50, 8)
(67, 8)(65, 8)

(20, 8)
(75, 8)
(64, 8)
(184, 8)
(389, 8)
(1041, 8)
(604, 8)
(806, 8)
(21, 8)
(83, 8)
(158, 8)
(39, 8)
(22, 8)
(21, 8)
(125, 8)
(447, 8)
(2074, 8)
(317, 8)
(362, 8)
(1143, 8)
(44, 8)
(101, 8)
(25, 8)
(28, 8)
(67, 8)
(68, 8)
(37, 8)
(416, 8)
(391, 8)
(460, 8)
(481, 8)
(876, 8)
(44, 8)
(16, 8)
(57, 8)
(50, 8)
(261, 8)
(70, 8)
(331, 8)
(433, 8)
(4342, 8)
(555, 8)(290, 8)

(3833, 8)
(25, 8)(39, 8)

(201, 8)
(80, 8)
(156, 8)
(33, 8)
(166, 8)
(29, 8)
(6883, 8)
(754, 8)
(855, 8)
(991, 8)
(28, 8)
(65, 8)(18, 8)

(16, 8)
(64, 8)
(27, 8)
(65, 8)
(440, 8)
(

(808, 8)
(896, 8)
(12, 8)
(72, 8)
(61, 8)
(44, 8)
(91, 8)
(1275, 8)
(52, 8)
(74, 8)
(1318, 8)
(1673, 8)
(3507, 8)(4389, 8)

(91, 8)
(30, 8)
(127, 8)
(113, 8)
(52, 8)
(619, 8)
(774, 8)
(44, 8)
(82, 8)
(26, 8)
(25, 8)
(509, 8)
(281, 8)
(970, 8)
(6147, 8)
(44, 8)
(18, 8)
(843, 8)
(325, 8)
(38, 8)
(13, 8)
(51, 8)
(8, 8)
(89, 8)
(19, 8)
(1552, 8)
(1012, 8)
(3655, 8)
(78, 8)
(155, 8)
(181, 8)
(48, 8)
(37, 8)
(269, 8)
(73, 8)
(5586, 8)
(131, 8)
(3120, 8)
(940, 8)
(384, 8)
(46, 8)
(157, 8)
(106, 8)
(13, 8)
(80, 8)
(127, 8)
(303, 8)
(47, 8)
(6422, 8)
(790, 8)
(508, 8)
(789, 8)
(137, 8)
(49, 8)
(11, 8)
(19, 8)
(252, 8)
(36, 8)
(1530, 8)
(58, 8)
(3099, 8)
(1647, 8)
(4243, 8)
(429, 8)
(74, 8)
(158, 8)
(128, 8)
(32, 8)
(98, 8)
(36, 8)
(26, 8)
(260, 8)
(1402, 8)
(106, 8)
(2492, 8)
(96, 8)
(127, 8)
(82, 8)
(46, 8)
(85, 8)
(817, 8)
(43, 8)
(84, 8)
(61, 8)
(1411, 8)
(2962, 8)
(135, 8)
(1893, 8)
(99, 8)
(39, 8)
(30, 8)
(40, 8)
(71, 8)
(45, 8)
(7, 8)
(170, 8)
(612, 8)
(40, 8)
(518, 8)
(63, 8)
(11, 8)
(19

(53, 8)
(55, 8)
(678, 8)
(719, 8)
(163, 8)
(84, 8)
(19, 8)
(1372, 8)
(1371, 8)
(367, 8)
(912, 8)
(167, 8)
(33, 8)
(131, 8)
(816, 8)
(752, 8)
(199, 8)
(42, 8)
(68, 8)
(327, 8)
(3665, 8)
(1070, 8)
(449, 8)
(33, 8)
(119, 8)
(66, 8)
(48, 8)
(460, 8)
(17, 8)
(22, 8)
(198, 8)
(763, 8)
(6184, 8)
(1750, 8)
(631, 8)
(79, 8)
(389, 8)
(80, 8)
(46, 8)
(113, 8)
(211, 8)
(23, 8)
(39, 8)
(167, 8)
(1683, 8)
(3675, 8)
(53, 8)
(98, 8)
(71, 8)
(754, 8)
(1053, 8)
(50, 8)
(57, 8)
(132, 8)
(6309, 8)
(482, 8)
(1004, 8)
(61, 8)
(47, 8)
(75, 8)
(573, 8)
(41, 8)
(1008, 8)
(59, 8)
(39, 8)
(802, 8)
(1635, 8)
(46, 8)
(218, 8)
(378, 8)
(19, 8)
(516, 8)
(1782, 8)
(3000, 8)
(115, 8)
(133, 8)
(78, 8)
(1840, 8)
(577, 8)
(20, 8)
(230, 8)
(31, 8)
(268, 8)
(173, 8)
(7045, 8)
(823, 8)
(446, 8)
(24, 8)
(22, 8)
(467, 8)
(2684, 8)
(33, 8)
(16, 8)
(25, 8)
(58, 8)
(578, 8)
(3000, 8)
(1460, 8)
(237, 8)
(15, 8)
(133, 8)
(165, 8)
(543, 8)
(29, 8)
(35, 8)
(1288, 8)
(19, 8)
(143, 8)
(1140, 8)
(165, 8)
(153, 8)
(875, 8)
(1289, 8)
(24

(70, 8)
(236, 8)
(56, 8)
(2, 8)
(1473, 8)
(1261, 8)
(1516, 8)
(48, 8)
(22, 8)
(655, 8)
(701, 8)
(76, 8)
(9, 8)
(40, 8)
(88, 8)
(15, 8)
(104, 8)
(1381, 8)
(1707, 8)
(47, 8)
(29, 8)
(3696, 8)
(142, 8)
(2178, 8)
(191, 8)
(80, 8)
(126, 8)
(18, 8)
(280, 8)
(1255, 8)
(4459, 8)
(72, 8)
(125, 8)
(17, 8)
(1059, 8)
(425, 8)
(86, 8)
(82, 8)
(63, 8)
(83, 8)
(210, 8)
(2211, 8)
(125, 8)
(112, 8)
(2471, 8)
(40, 8)
(2776, 8)
(103, 8)
(98, 8)
(2363, 8)
(41, 8)
(93, 8)
(25, 8)
(53, 8)
(376, 8)
(1765, 8)
(442, 8)
(111, 8)
(95, 8)
(33, 8)
(2261, 8)
(2170, 8)
(82, 8)
(18, 8)
(207, 8)
(69, 8)
(201, 8)
(607, 8)
(893, 8)
(21, 8)
(7, 8)
(55, 8)
(495, 8)
(22, 8)
(21, 8)
(1905, 8)
(103, 8)
(18, 8)
(133, 8)
(1893, 8)
(1162, 8)
(119, 8)
(203, 8)
(69, 8)
(31, 8)
(50, 8)
(117, 8)
(466, 8)
(142, 8)
(31, 8)
(315, 8)
(968, 8)
(162, 8)
(1033, 8)
(116, 8)
(77, 8)
(152, 8)
(93, 8)
(1048, 8)
(2954, 8)
(27, 8)
(33, 8)
(85, 8)
(42, 8)
(358, 8)
(638, 8)
(78, 8)
(296, 8)
(56, 8)
(144, 8)
(3830, 8)
(719, 8)
(60, 8)
(7, 8)
(670,

(167, 8)
(10, 8)
(823, 8)
(414, 8)
(450, 8)
(211, 8)
(348, 8)
(26, 8)
(167, 8)
(160, 8)
(45, 8)
(60, 8)
(278, 8)
(90, 8)
(306, 8)
(678, 8)
(1204, 8)
(324, 8)
(96, 8)
(819, 8)
(1629, 8)
(3183, 8)
(27, 8)
(136, 8)
(55, 8)
(158, 8)
(627, 8)
(1144, 8)
(2251, 8)
(3319, 8)
(129, 8)
(58, 8)
(178, 8)
(53, 8)
(1059, 8)
(698, 8)
(26, 8)
(194, 8)
(997, 8)
(1400, 8)
(3748, 8)
(427, 8)
(26, 8)
(897, 8)
(25, 8)
(37, 8)
(209, 8)
(297, 8)
(70, 8)
(12, 8)
(229, 8)
(25, 8)
(256, 8)
(371, 8)
(1143, 8)
(2057, 8)
(10, 8)
(20, 8)
(40, 8)
(19, 8)
(200, 8)
(858, 8)
(101, 8)
(114, 8)
(1308, 8)
(1491, 8)
(761, 8)
(3709, 8)
(9, 8)
(57, 8)
(11, 8)
(41, 8)
(826, 8)
(233, 8)
(333, 8)
(173, 8)
(137, 8)
(224, 8)
(585, 8)
(565, 8)
(25, 8)
(21, 8)
(25, 8)
(489, 8)
(98, 8)
(143, 8)
(120, 8)
(77, 8)
(310, 8)
(44, 8)
(76, 8)
(1598, 8)
(619, 8)
(1494, 8)
(29, 8)
(49, 8)
(280, 8)
(233, 8)
(96, 8)
(498, 8)
(196, 8)
(102, 8)
(27, 8)
(262, 8)
(5055, 8)
(812, 8)
(20, 8)
(164, 8)
(1159, 8)
(2331, 8)
(108, 8)
(84, 8)
(156, 8)
(65

(151, 8)
(227, 8)
(49, 8)
(669, 8)
(414, 8)
(44, 8)
(239, 8)
(3399, 8)
(1050, 8)
(226, 8)
(9, 8)
(191, 8)
(126, 8)
(36, 8)
(113, 8)
(352, 8)
(1244, 8)
(93, 8)
(1466, 8)
(206, 8)
(48, 8)
(127, 8)
(67, 8)
(250, 8)
(34, 8)
(1316, 8)
(77, 8)
(70, 8)
(43, 8)
(4952, 8)
(1784, 8)
(501, 8)
(52, 8)
(112, 8)
(75, 8)
(144, 8)
(102, 8)
(506, 8)
(74, 8)
(358, 8)
(147, 8)
(2874, 8)
(2083, 8)
(456, 8)
(116, 8)
(71, 8)
(92, 8)
(188, 8)
(113, 8)
(851, 8)
(48, 8)
(51, 8)
(34, 8)
(196, 8)
(3276, 8)
(15, 8)
(45, 8)
(11, 8)(336, 8)

(88, 8)
(77, 8)
(21, 8)
(311, 8)
(460, 8)
(214, 8)
(2719, 8)
(390, 8)
(53, 8)
(93, 8)
(38, 8)
(339, 8)
(81, 8)
(228, 8)
(43, 8)
(318, 8)
(25, 8)
(950, 8)
(1360, 8)
(41, 8)
(30, 8)
(92, 8)
(60, 8)
(300, 8)
(658, 8)
(2134, 8)
(45, 8)
(59, 8)
(493, 8)
(130, 8)
(1958, 8)
(71, 8)
(85, 8)
(23, 8)
(183, 8)
(200, 8)
(757, 8)
(46, 8)
(88, 8)
(77, 8)
(173, 8)
(100, 8)
(2555, 8)
(58, 8)
(113, 8)
(207, 8)
(29, 8)
(353, 8)
(611, 8)
(234, 8)
(27, 8)
(74, 8)
(86, 8)
(1179, 8)
(366, 8)
(32, 8)

(1059, 8)
(140, 8)
(584, 8)
(455, 8)
(15, 8)
(160, 8)
(848, 8)
(63, 8)
(291, 8)
(162, 8)
(16, 8)
(604, 8)
(122, 8)
(120, 8)(120, 8)

(47, 8)
(199, 8)
(2242, 8)
(204, 8)
(105, 8)
(128, 8)
(6, 8)
(95, 8)
(990, 8)
(505, 8)
(23, 8)
(332, 8)
(28, 8)
(244, 8)
(236, 8)
(47, 8)
(983, 8)
(142, 8)
(26, 8)
(88, 8)
(476, 8)
(68, 8)
(64, 8)
(575, 8)
(31, 8)
(148, 8)
(383, 8)
(120, 8)
(79, 8)
(92, 8)
(118, 8)
(269, 8)
(268, 8)
(92, 8)
(654, 8)
(61, 8)
(181, 8)
(34, 8)
(65, 8)
(354, 8)
(77, 8)
(681, 8)
(45, 8)
(79, 8)
(53, 8)
(34, 8)
(117, 8)
(1120, 8)
(381, 8)
(118, 8)
(50, 8)
(117, 8)
(143, 8)
(3230, 8)
(29, 8)
(562, 8)
(129, 8)
(44, 8)
(23, 8)
(219, 8)
(95, 8)
(5, 8)
(41, 8)
(27, 8)
(416, 8)
(82, 8)
(78, 8)
(911, 8)
(21, 8)
(83, 8)
(98, 8)
(27, 8)
(14, 8)
(55, 8)
(50, 8)
(18, 8)
(53, 8)
(17, 8)
(89, 8)
(5786, 8)
(23, 8)
(226, 8)
(96, 8)
(56, 8)
(51, 8)
(300, 8)
(103, 8)
(37, 8)
(170, 8)
(11, 8)
(63, 8)
(919, 8)
(45, 8)
(256, 8)
(2244, 8)
(11, 8)
(31, 8)
(29, 8)
(111, 8)
(755, 8)
(4543, 8)
(36, 8)


id
5a2a61bf588c13481c229d1e    None
5a2a61bf588c13481c229d1f    None
5a2a61bf588c13481c229d20    None
5a2a61bf588c13481c229d21    None
5a2a61bf588c13481c229d22    None
5a2a61bf588c13481c229d23    None
5a2a61bf588c13481c229d24    None
5a2a61bf588c13481c229d25    None
5a2a61bf588c13481c229d26    None
5a2a61bf588c13481c229d27    None
5a2a61bf588c13481c229d28    None
5a2a61bf588c13481c229d29    None
5a2a61bf588c13481c229d2a    None
5a2a61bf588c13481c229d2b    None
5a2a61bf588c13481c229d2c    None
5a2a61bf588c13481c229d2d    None
5a2a61bf588c13481c229d2e    None
5a2a61bf588c13481c229d2f    None
5a2a61bf588c13481c229d30    None
5a2a61bf588c13481c229d31    None
5a2a61bf588c13481c229d32    None
5a2a61bf588c13481c229d33    None
5a2a61bf588c13481c229d34    None
5a2a61bf588c13481c229d35    None
5a2a61bf588c13481c229d36    None
5a2a61bf588c13481c229d37    None
5a2a61bf588c13481c229d38    None
5a2a61bf588c13481c229d39    None
5a2a61bf588c13481c229d3a    None
5a2a61bf588c13481c229d3b    None
       

### 뉴스의 댓글 파일을 하나로 합침

In [11]:
%%time
pre_outfile_daum = outcome_predata + 'predata_for_daum_news_comment.csv'
print (os.path.isfile(pre_outfile_daum))
if os.path.isfile(pre_outfile_daum):
    dfForDaumCommentsAnalysis = pd.read_csv(pre_outfile_daum, header= 0, encoding='utf-8')
else:
    dfForDaumCommentsAnalysis = pd.DataFrame()
    for idx2 in reDaumData.index:
        commentFile = os.path.join(daumCommentsPath, idx2+'.csv')
        tempDf = pd.read_csv(commentFile, header=0, encoding='utf-8')
        # comment의 mongoDB id를 News의 mongoDB id로 변환 뒤에 분석의 용이함을 위해서
        tempDf._id = idx2
        dfForDaumCommentsAnalysis = dfForDaumCommentsAnalysis.append(tempDf)
    print (dfForDaumCommentsAnalysis.shape)
    dfForDaumCommentsAnalysis.to_csv(pre_outfile_daum, header = True, encoding = 'utf-8')

True
Wall time: 24.8 s


In [12]:
dfForDaumCommentsAnalysis.shape

(4924599, 9)

In [13]:
dfForDaumCommentsAnalysis.dropna(inplace = True)

In [14]:
dfForDaumCommentsAnalysis.shape

(4924599, 9)

In [ ]:
pre_outfile_daum_filtered = outcome_predata + 'filtered_predata_for_daum_news_comment.csv'
dfForDaumCommentsAnalysis.to_csv(pre_outfile_daum_filtered, header = True, encoding = 'utf-8')